In [2]:
from dimod.generators import and_gate

bqm = and_gate('in1','in2','out')
bqm

BinaryQuadraticModel({'in1': 0.0, 'in2': 0.0, 'out': 3.0}, {('in2', 'in1'): 1.0, ('out', 'in1'): -2.0, ('out', 'in2'): -2.0}, 0.0, 'BINARY')

In [3]:
from dwave.system import LeapHybridSampler
sampler = LeapHybridSampler()    
answer = sampler.sample(bqm)   
print(answer)    

  in1 in2 out energy num_oc.
0   1   0   0    0.0       1
['BINARY', 1 rows, 1 samples, 3 variables]


In [4]:
bqm.to_qubo()

({('in2', 'in1'): 1.0,
  ('out', 'in1'): -2.0,
  ('out', 'in2'): -2.0,
  ('in1', 'in1'): 0.0,
  ('in2', 'in2'): 0.0,
  ('out', 'out'): 3.0},
 0.0)

## Prova number partitioning

Ho usato pyqubo, prova direttamente con dimod

*nota*: qubo è una bqm.binary per dimod, tutto qui. quindi poi risolvi come una bqm

In [5]:
from pyqubo import Array

x = Array.create('x', shape = (1,5), vartype = 'BINARY')
x

Array([[Binary('x[0][0]'), Binary('x[0][1]'), Binary('x[0][2]'), Binary('x[0][3]'), Binary('x[0][4]')]])

In [6]:
num = [1,2,3,4,10]
tot = sum(num)
tot

20

In [7]:
import numpy as np

Q = np.zeros((5,5))

for i in range(5):
    for j in range(i,5):
        if i == j: 
            Q[i][j] = num[i]*(num[i] - tot)
        else:
            Q[i][j] = num[i]*num[j]
            Q[j][i] = num[i]*num[j]

Q

array([[ -19.,    2.,    3.,    4.,   10.],
       [   2.,  -36.,    6.,    8.,   20.],
       [   3.,    6.,  -51.,   12.,   30.],
       [   4.,    8.,   12.,  -64.,   40.],
       [  10.,   20.,   30.,   40., -100.]])

In [8]:
#Arr_Q = Array(Q)
#Arr_Q

In [9]:
#x = x_t.T
#x.shape

In [10]:
'''
ph = np.zeros((5,1))
for i in range(5): ph[i] = Q[0][i]
print(ph.shape)
sum(x*ph) # non va bene per creare il model
'''

'\nph = np.zeros((5,1))\nfor i in range(5): ph[i] = Q[0][i]\nprint(ph.shape)\nsum(x*ph) # non va bene per creare il model\n'

In [11]:
ph_model = 0

for i in range(5):
    for j in range(5):
        ph_model = ph_model + x[0,i]*Q[i][j]*x[0,j]

model = ph_model.compile()
model

In [12]:
qubo_model = model.to_qubo()
print(qubo_model)
# vedi come stampare carino

({('x[0][4]', 'x[0][3]'): 80.0, ('x[0][3]', 'x[0][1]'): 16.0, ('x[0][0]', 'x[0][0]'): -19.0, ('x[0][3]', 'x[0][3]'): -64.0, ('x[0][1]', 'x[0][0]'): 4.0, ('x[0][4]', 'x[0][1]'): 40.0, ('x[0][4]', 'x[0][2]'): 60.0, ('x[0][3]', 'x[0][2]'): 24.0, ('x[0][2]', 'x[0][2]'): -51.0, ('x[0][3]', 'x[0][0]'): 8.0, ('x[0][2]', 'x[0][0]'): 6.0, ('x[0][2]', 'x[0][1]'): 12.0, ('x[0][4]', 'x[0][4]'): -100.0, ('x[0][1]', 'x[0][1]'): -36.0, ('x[0][4]', 'x[0][0]'): 20.0}, 0.0)


In [22]:
import neal

bqm = model.to_bqm()
print(bqm)
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
samples = model.decode_sampleset(sampleset)
best_sample = min(samples, key=lambda s: s.energy)
print(best_sample.sample) 

BinaryQuadraticModel({'x[0][1]': -36.0, 'x[0][0]': -19.0, 'x[0][4]': -100.0, 'x[0][2]': -51.0, 'x[0][3]': -64.0}, {('x[0][0]', 'x[0][1]'): 4.0, ('x[0][4]', 'x[0][1]'): 40.0, ('x[0][4]', 'x[0][0]'): 20.0, ('x[0][2]', 'x[0][1]'): 12.0, ('x[0][2]', 'x[0][0]'): 6.0, ('x[0][2]', 'x[0][4]'): 60.0, ('x[0][3]', 'x[0][1]'): 16.0, ('x[0][3]', 'x[0][0]'): 8.0, ('x[0][3]', 'x[0][4]'): 80.0, ('x[0][3]', 'x[0][2]'): 24.0}, 0.0, 'BINARY')
{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1}


che in effetti è corretto: prendo S_1 = {x[0][4]} e S_2 come i primi 4 elementi

In [14]:
# simulated annealing
from dwave.system import LeapHybridSampler
sampler = LeapHybridSampler()    
answer = sampler.sample(bqm)   
print(answer) 

  x[0][0] x[0][1] x[0][2] x[0][3] x[0][4] energy num_oc.
0       1       1       1       1       0 -100.0       1
['BINARY', 1 rows, 1 samples, 5 variables]


# Prova number partitioning

con dwave direttamente

In [15]:
import numpy as np
import dimod as dd

num = [1,2,3,4,10]
tot = sum(num)

Q = np.zeros((5,5))

for i in range(5):
    for j in range(i,5):
        if i == j: 
            Q[i][j] = num[i]*(num[i] - tot)
        else:
            Q[i][j] = num[i]*num[j]
            Q[j][i] = num[i]*num[j]

print(Q)

[[ -19.    2.    3.    4.   10.]
 [   2.  -36.    6.    8.   20.]
 [   3.    6.  -51.   12.   30.]
 [   4.    8.   12.  -64.   40.]
 [  10.   20.   30.   40. -100.]]


In [16]:
qubo = dict()

for i in range(5):
    for j in range(5):
        qubo[(i,j)] = Q[i][j]

print(qubo)

bqm_wave = dd.BQM.from_qubo(qubo)

bqm_wave

{(0, 0): -19.0, (0, 1): 2.0, (0, 2): 3.0, (0, 3): 4.0, (0, 4): 10.0, (1, 0): 2.0, (1, 1): -36.0, (1, 2): 6.0, (1, 3): 8.0, (1, 4): 20.0, (2, 0): 3.0, (2, 1): 6.0, (2, 2): -51.0, (2, 3): 12.0, (2, 4): 30.0, (3, 0): 4.0, (3, 1): 8.0, (3, 2): 12.0, (3, 3): -64.0, (3, 4): 40.0, (4, 0): 10.0, (4, 1): 20.0, (4, 2): 30.0, (4, 3): 40.0, (4, 4): -100.0}


BinaryQuadraticModel({0: -19.0, 1: -36.0, 2: -51.0, 3: -64.0, 4: -100.0}, {(1, 0): 4.0, (2, 0): 6.0, (2, 1): 12.0, (3, 0): 8.0, (3, 1): 16.0, (3, 2): 24.0, (4, 0): 20.0, (4, 1): 40.0, (4, 2): 60.0, (4, 3): 80.0}, 0.0, 'BINARY')

In [17]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampleset = EmbeddingComposite(DWaveSampler()).sample(bqm_wave, num_reads=100) # submit to a QPU # DAVVERO?! # SIIIIIIIIIIIIIII
sampleset

SampleSet(rec.array([([1, 1, 1, 1, 0], -100.,  7, 0.),
           ([0, 0, 0, 0, 1], -100., 25, 0.),
           ([0, 1, 1, 1, 0],  -99.,  8, 0.),
           ([1, 0, 0, 0, 1],  -99., 28, 0.),
           ([0, 1, 0, 0, 1],  -96., 18, 0.),
           ([1, 0, 1, 1, 0],  -96.,  4, 0.),
           ([0, 0, 1, 1, 0],  -91.,  1, 0.),
           ([1, 1, 0, 1, 0],  -91.,  3, 0.),
           ([1, 1, 0, 0, 1],  -91.,  2, 0.),
           ([0, 0, 1, 0, 1],  -91.,  2, 0.),
           ([0, 1, 0, 1, 0],  -84.,  2, 0.)],
          dtype=[('sample', 'i1', (5,)), ('energy', '<f8'), ('num_occurrences', '<i4'), ('chain_break_fraction', '<f8')]), Variables([0, 1, 2, 3, 4]), {'timing': {'qpu_sampling_time': 8196.0, 'qpu_anneal_time_per_sample': 20.0, 'qpu_readout_time_per_sample': 41.42, 'qpu_access_time': 23956.37, 'qpu_access_overhead_time': 1519.63, 'qpu_programming_time': 15760.37, 'qpu_delay_time_per_sample': 20.54, 'total_post_processing_time': 974.0, 'post_processing_overhead_time': 974.0}, 'problem_id': 

In [21]:
EmbeddingComposite(DWaveSampler()).sample_qubo(qubo, num_reads=100)

# superfluo
# sample_qubo prende il qubo, lo trasforma in bqm e poi risolve
# posso farlo anche io a priori quindi

SampleSet(rec.array([([0, 0, 0, 0, 1], -100., 14, 0.),
           ([1, 1, 1, 1, 0], -100., 12, 0.),
           ([1, 0, 0, 0, 1],  -99., 24, 0.),
           ([0, 1, 1, 1, 0],  -99.,  8, 0.),
           ([0, 1, 0, 0, 1],  -96., 10, 0.),
           ([1, 0, 1, 1, 0],  -96., 16, 0.),
           ([1, 1, 0, 1, 0],  -91.,  2, 0.),
           ([0, 0, 1, 0, 1],  -91.,  3, 0.),
           ([1, 1, 0, 0, 1],  -91.,  5, 0.),
           ([0, 0, 0, 1, 1],  -84.,  2, 0.),
           ([1, 1, 1, 0, 0],  -84.,  2, 0.),
           ([0, 1, 0, 1, 0],  -84.,  1, 0.),
           ([1, 0, 0, 1, 1],  -75.,  1, 0.)],
          dtype=[('sample', 'i1', (5,)), ('energy', '<f8'), ('num_occurrences', '<i4'), ('chain_break_fraction', '<f8')]), Variables([0, 1, 2, 3, 4]), {'timing': {'qpu_sampling_time': 9876.0, 'qpu_anneal_time_per_sample': 20.0, 'qpu_readout_time_per_sample': 58.22, 'qpu_access_time': 25636.37, 'qpu_access_overhead_time': 580.63, 'qpu_programming_time': 15760.37, 'qpu_delay_time_per_sample': 20.54, 'to

In [71]:
from dwave.system import LeapHybridSampler
sampler = LeapHybridSampler(solver={'category': 'hybrid'})    
sampleset_hybrid = sampler.sample(bqm_wave)   
print(sampleset_hybrid)    
sampleset_hybrid.info

   0  1  2  3  4 energy num_oc.
0  0  0  0  0  1 -100.0       1
['BINARY', 1 rows, 1 samples, 5 variables]


{'qpu_access_time': 80305,
 'charge_time': 2989572,
 'run_time': 2989572,
 'problem_id': '91ac58fa-bf65-4dcc-9e3f-e99708425721'}

In [73]:
answer = sampler.sample_qubo(qubo)
print(answer)

   0  1  2  3  4 energy num_oc.
0  0  0  0  0  1 -100.0       1
['BINARY', 1 rows, 1 samples, 5 variables]


Io non ho capito perchè mi dava problemi con la matrice e mi ha fatto fare tremila passaggi intermedi QUANDO BASTAVA PASSARE LA MATRICE ALLA FUNZIONE, VISTO CHE Q RAPPRESENTA GIà IL PROBLEMA IN MODELLO QUBO. 

io boh

sintassi da usare: dd.BQM.from_qubo(Q)